In [1]:
import os
import errno
import argparse
import sys
import pickle

import numpy as np
from tensorflow.keras.models import load_model

from data_utils import load_MNIST_data, load_EMNIST_data, generate_EMNIST_writer_based_data, generate_partial_data
from skd import FedMD
from Neural_Networks import train_models, cnn_2layer_fc_model, cnn_3layer_fc_model

from tensorflow.keras.models import clone_model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

from sklearn.model_selection import train_test_split

2023-05-17 17:56:18.139824: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-17 17:56:18.160929: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-17 17:56:18.464972: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
conf_file = os.path.abspath("conf/EMNIST_imbalance_conf.json")

In [3]:
def poisoning(targets):

    for idx in range(len(targets)):
        if targets[idx] == 10:
            targets[idx] = 14
        elif targets[idx] == 11:
            targets[idx] = 12
        elif targets[idx] == 12:
            targets[idx] = 11
        elif targets[idx] == 13:
            targets[idx] = 15
        elif targets[idx] == 14:
            targets[idx] = 10
        elif targets[idx] == 15:
            targets[idx] = 13

    return targets

In [4]:
CANDIDATE_MODELS = {"2_layer_CNN": cnn_2layer_fc_model, 
                    "3_layer_CNN": cnn_3layer_fc_model} 

In [5]:
with open(conf_file, "r") as f:
    conf_dict = eval(f.read())
    
    #n_classes = conf_dict["n_classes"]
    model_config = conf_dict["models"]
    pre_train_params = conf_dict["pre_train_params"]
    model_saved_dir = conf_dict["model_saved_dir"]
    model_saved_names = conf_dict["model_saved_names"]
    is_early_stopping = conf_dict["early_stopping"]
    public_classes = conf_dict["public_classes"]
    private_classes = conf_dict["private_classes"]
    n_classes = len(public_classes) + len(private_classes)
    
    emnist_data_dir = conf_dict["EMNIST_dir"]    
    N_parties = conf_dict["N_parties"]
    # N_samples_per_class = conf_dict["N_samples_per_class"]
    N_samples_per_class = 5  # ssfd
    
    N_rounds = conf_dict["N_rounds"]
    N_alignment = conf_dict["N_alignment"]
    N_private_training_round = conf_dict["N_private_training_round"]
    private_training_batchsize = conf_dict["private_training_batchsize"]
    N_logits_matching_round = conf_dict["N_logits_matching_round"]
    logits_matching_batchsize = conf_dict["logits_matching_batchsize"]
    
    
    result_save_dir = conf_dict["result_save_dir"]


del conf_dict, conf_file

In [6]:
X_train_MNIST, y_train_MNIST, X_test_MNIST, y_test_MNIST \
= load_MNIST_data(standarized = True, verbose = True)

public_dataset = {"X": X_train_MNIST, "y": y_train_MNIST}


X_train_EMNIST, y_train_EMNIST, X_test_EMNIST, y_test_EMNIST, \
writer_ids_train_EMNIST, writer_ids_test_EMNIST \
= load_EMNIST_data(emnist_data_dir,
                    standarized = True, verbose = True)

y_train_EMNIST += len(public_classes)
y_test_EMNIST += len(public_classes)

MNIST dataset ... 
X_train shape : (60000, 28, 28)
X_test shape : (10000, 28, 28)
y_train shape : (60000,)
y_test shape : (10000,)
EMNIST-letter dataset ... 
X_train shape : (124800, 28, 28)
X_test shape : (20800, 28, 28)
y_train shape : (124800,)
y_test shape : (20800,)


In [7]:
X_train_EMNIST.shape

(124800, 28, 28)

In [14]:
np.unique(np.unique(writer_ids_train_EMNIST, return_counts=True)[1], return_counts=True)

(array([  1,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
         15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
         41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
         54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
         67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
         80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  92,  93,
         94,  95,  96, 104, 121]),
 array([  3,   2,   3,   7,   3,  10,  14,  13,  11,  44,  34,  56,  67,
        135, 159, 192, 205, 213, 236, 203, 174, 121, 109,  61,  49,  23,
         17,  15,   6,   9,  12,  10,  15,  19,   9,  19,  14,  21,  24,
         21,  11,  14,  19,  22,  30,  27,  19,  18,  30,  31,  38,  29,
         28,  33,  34,  45,  34,  34,  41,  35,  38,  43,  33,  37,  38,
         36,  29,  42,  38,  36,  35,  34,  26,  17,  25,  24,  23,  13,
         13,  11

In [9]:
#generate private data
private_data, total_private_data\
=generate_EMNIST_writer_based_data(X_train_EMNIST, y_train_EMNIST,
                                    writer_ids_train_EMNIST,
                                    N_parties = N_parties, 
                                    classes_in_use = private_classes, 
                                    N_priv_data_min = N_samples_per_class * len(private_classes)
                                    )

X_tmp, y_tmp = generate_partial_data(X = X_test_EMNIST, y= y_test_EMNIST, 
                                        class_in_use = private_classes, verbose = True)
private_test_data = {"X": X_tmp, "y": y_tmp}
del X_tmp, y_tmp

ValueError: Cannot take a larger sample than population when 'replace=False'

In [8]:
private_data[0]["X"].shape

(20, 28, 28)

In [9]:
private_test_data["X"].shape

(4800, 28, 28)

In [10]:
# Poisoning

poison_devices = [0,1,2,3]

for device in poison_devices:
    private_data[device]['y'] = poisoning(private_data[device]['y'])
    print(private_data[device]['y'])

[15 15 12 14 11 14 10 10 10 15 10 10 11 15 12 10 15 10 10 10]
[10 15 10 12 12 14 10 13 10 11 13 14 13 15 11 12 15 11 14 15 14 11 15]
[14 15 10 10 13 15 15 13 10 15 13 11 12 13 10 12 12 14 10 11]
[10 10 10 12 10 15 14 14 13 10 13 10 15 14 10 13 15 15 12 12 10 11 15 11
 11]


In [11]:
parties = []
if model_saved_dir is None:
    for i, item in enumerate(model_config):
        model_name = item["model_type"]
        model_params = item["params"]
        tmp = CANDIDATE_MODELS[model_name](n_classes=n_classes, 
                                            input_shape=(28,28),
                                            **model_params)
        print("model {0} : {1}".format(i, model_saved_names[i]))
        print(tmp.summary())
        parties.append(tmp)
        
        del model_name, model_params, tmp
    #END FOR LOOP
    pre_train_result = train_models(parties, 
                                    X_train_MNIST, y_train_MNIST, 
                                    X_test_MNIST, y_test_MNIST,
                                    save_dir = model_saved_dir, save_names = model_saved_names,
                                    early_stopping = is_early_stopping,
                                    **pre_train_params
                                    )
else:
    dpath = os.path.abspath(model_saved_dir)
    model_names = os.listdir(dpath)
    for name in model_names:
        tmp = None
        tmp = load_model(os.path.join(dpath ,name))
        parties.append(tmp)

del  X_train_MNIST, y_train_MNIST, X_test_MNIST, y_test_MNIST, \
X_train_EMNIST, y_train_EMNIST, X_test_EMNIST, y_test_EMNIST, writer_ids_train_EMNIST, writer_ids_test_EMNIST

2023-05-12 08:54:48.337458: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-12 08:54:48.352534: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-12 08:54:48.352609: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

model 0 : CNN_128_256
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 128)       1280      
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 128)      512       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 28, 28, 128)       0         
                                                                 
 dropout (Dropout)           (None, 28, 28, 128)       0         
                                       

None
model 1 : CNN_128_384
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 reshape_1 (Reshape)         (None, 28, 28, 1)         0         
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       1280      
                                                                 
 batch_normalization_2 (Batc  (None, 28, 28, 128)      512       
 hNormalization)                                                 
                                                                 
 activation_3 (Activation)   (None, 28, 28, 128)       0         
                                                                 
 dropout_2 (Dropout)         (None, 28, 28, 128)       0         
                                

None
model 2 : CNN_128_512
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 reshape_2 (Reshape)         (None, 28, 28, 1)         0         
                                                                 
 conv2d_4 (Conv2D)           (None, 28, 28, 128)       1280      
                                                                 
 batch_normalization_4 (Batc  (None, 28, 28, 128)      512       
 hNormalization)                                                 
                                                                 
 activation_6 (Activation)   (None, 28, 28, 128)       0         
                                                                 
 dropout_4 (Dropout)         (None, 28, 28, 128)       0         
                                

None
model 3 : CNN_256_256
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 reshape_3 (Reshape)         (None, 28, 28, 1)         0         
                                                                 
 conv2d_6 (Conv2D)           (None, 28, 28, 256)       2560      
                                                                 
 batch_normalization_6 (Batc  (None, 28, 28, 256)      1024      
 hNormalization)                                                 
                                                                 
 activation_9 (Activation)   (None, 28, 28, 256)       0         
                                                                 
 dropout_6 (Dropout)         (None, 28, 28, 256)       0         
                                

...

469/469 [==============================] - 5s 8ms/step - loss: 0.1450 - accuracy: 0.9639 - val_loss: 0.1784 - val_accuracy: 0.9789
Epoch 2/20
469/469 [==============================] - 3s 7ms/step - loss: 0.0700 - accuracy: 0.9846 - val_loss: 0.0670 - val_accuracy: 0.9844
Epoch 3/20
469/469 [==============================] - 3s 7ms/step - loss: 0.0577 - accuracy: 0.9880 - val_loss: 0.0555 - val_accuracy: 0.9882
Epoch 4/20
469/469 [==============================] - 3s 7ms/step - loss: 0.0513 - accuracy: 0.9893 - val_loss: 0.0497 - val_accuracy: 0.9890
Epoch 5/20
469/469 [==============================] - 3s 7ms/step - loss: 0.0468 - accuracy: 0.9902 - val_loss: 0.0666 - val_accuracy: 0.9832
Epoch 6/20
469/469 [==============================] - 3s 7ms/step - loss: 0.0439 - accuracy: 0.9917 - val_loss: 0.0421 - val_accuracy: 0.9907
Epoch 7/20
469/469 [==============================] - 3s 7ms/step - loss: 0.0425 - accuracy: 0.9916 - val_loss: 0.0468 - val_accuracy: 0.9907
Epoch 8/20
469/46

2023-05-12 08:59:39.019224: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_8/dropout_19/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


469/469 [==============================] - 4s 7ms/step - loss: 0.1901 - accuracy: 0.9506 - val_loss: 0.2800 - val_accuracy: 0.9728
Epoch 2/20
469/469 [==============================] - 3s 6ms/step - loss: 0.0853 - accuracy: 0.9811 - val_loss: 0.0544 - val_accuracy: 0.9887
Epoch 3/20
469/469 [==============================] - 3s 6ms/step - loss: 0.0675 - accuracy: 0.9857 - val_loss: 0.0532 - val_accuracy: 0.9890
Epoch 4/20
469/469 [==============================] - 3s 6ms/step - loss: 0.0603 - accuracy: 0.9873 - val_loss: 0.0544 - val_accuracy: 0.9877
Epoch 5/20
469/469 [==============================] - 3s 6ms/step - loss: 0.0557 - accuracy: 0.9879 - val_loss: 0.0552 - val_accuracy: 0.9878
Training model  9
Epoch 1/20


2023-05-12 08:59:54.947135: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_9/dropout_22/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


469/469 [==============================] - 4s 6ms/step - loss: 0.1707 - accuracy: 0.9563 - val_loss: 0.1921 - val_accuracy: 0.9811
Epoch 2/20
469/469 [==============================] - 3s 6ms/step - loss: 0.0777 - accuracy: 0.9826 - val_loss: 0.0542 - val_accuracy: 0.9893
Epoch 3/20
469/469 [==============================] - 3s 6ms/step - loss: 0.0642 - accuracy: 0.9861 - val_loss: 0.0523 - val_accuracy: 0.9884
Epoch 4/20
469/469 [==============================] - 3s 6ms/step - loss: 0.0580 - accuracy: 0.9878 - val_loss: 0.0552 - val_accuracy: 0.9875
Epoch 5/20
469/469 [==============================] - 3s 6ms/step - loss: 0.0536 - accuracy: 0.9887 - val_loss: 0.0456 - val_accuracy: 0.9907
Epoch 6/20
469/469 [==============================] - 3s 6ms/step - loss: 0.0504 - accuracy: 0.9897 - val_loss: 0.0443 - val_accuracy: 0.9908
Epoch 7/20
469/469 [==============================] - 3s 6ms/step - loss: 0.0488 - accuracy: 0.9900 - val_loss: 0.0486 - val_accuracy: 0.9896
Epoch 8/20
469/46

In [12]:
for i in range(10):
    print("model ", i)

    device = i

    X_train = private_data[device]['X']
    y_train = private_data[device]['y']

    print(X_train.shape, y_train.shape)
    print(y_train)

model  0
(20, 28, 28) (20,)
[15 15 12 14 11 14 10 10 10 15 10 10 11 15 12 10 15 10 10 10]
model  1
(23, 28, 28) (23,)
[10 15 10 12 12 14 10 13 10 11 13 14 13 15 11 12 15 11 14 15 14 11 15]
model  2
(20, 28, 28) (20,)
[14 15 10 10 13 15 15 13 10 15 13 11 12 13 10 12 12 14 10 11]
model  3
(25, 28, 28) (25,)
[10 10 10 12 10 15 14 14 13 10 13 10 15 14 10 13 15 15 12 12 10 11 15 11
 11]
model  4
(19, 28, 28) (19,)
[13 15 13 15 14 11 10 12 11 13 11 15 13 13 10 11 13 12 10]
model  5
(22, 28, 28) (22,)
[10 15 10 13 10 14 13 13 13 13 14 13 14 13 14 15 14 12 11 12 13 11]
model  6
(18, 28, 28) (18,)
[13 15 12 14 15 11 12 13 12 11 10 12 10 10 10 11 13 14]
model  7
(19, 28, 28) (19,)
[10 14 11 13 10 11 14 15 14 11 15 12 12 14 15 14 10 12 11]
model  8
(20, 28, 28) (20,)
[15 12 10 10 14 11 14 11 11 10 14 12 15 10 13 12 13 15 13 10]
model  9
(22, 28, 28) (22,)
[14 11 10 10 14 11 11 10 14 15 10 13 13 13 12 10 15 11 13 13 15 14]


In [13]:
import warnings

warnings.filterwarnings('ignore')

In [14]:
# Label propagation

for i in range(10):
    print("model ", i)

    device = i

    X_train = private_data[device]['X']
    y_train = private_data[device]['y']

    X_lab, X_unlab, y_lab, y_unlab = train_test_split(X_train, y_train, test_size=.5, random_state=1)

    model_A_twin = None
    model_A_twin = clone_model(parties[i]) # load private model
    model_A_twin.set_weights(parties[i].get_weights())
    model_A_twin.compile(optimizer=tf.keras.optimizers.Adam(lr = 1e-3), 
                            loss = "sparse_categorical_crossentropy",
                            metrics = ["accuracy"])

    print("Semi-supervised training ... ")        

    # train private models with private data
    model_A_twin.fit(X_lab, y_lab,
                        batch_size = 32, epochs = 25, shuffle=True, verbose = 0,
                        validation_data = [private_test_data["X"], private_test_data["y"]],
                        callbacks=[EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=10)]
                    )

    print("Semi-supervised training done")

    y_pred = model_A_twin.predict(X_unlab).argmax(axis=1)

    X_mixed = np.concatenate((X_lab, X_unlab))
    y_mixed = np.concatenate((y_lab, y_pred))

    private_data[device]['X'] = X_mixed
    private_data[device]['y'] = y_mixed

    del model_A_twin, X_mixed, y_mixed, X_lab, X_unlab, y_lab, y_unlab, X_train, y_train

model  0
Semi-supervised training ... 


2023-05-12 09:04:31.750127: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Semi-supervised training done
1/1 [==============================] - 0s 40ms/step


model  1
Semi-supervised training ... 


2023-05-12 09:04:34.637714: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/dropout_2/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


...

2023-05-12 09:05:04.031892: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_8/dropout_19/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Semi-supervised training done
1/1 [==============================] - 0s 35ms/step


model  9
Semi-supervised training ... 


2023-05-12 09:05:07.854400: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_9/dropout_22/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Semi-supervised training done
1/1 [==============================] - 0s 36ms/step


In [15]:
fedmd = FedMD(parties, 
                public_dataset = public_dataset,
                private_data = private_data, 
                total_private_data = total_private_data,
                private_test_data = private_test_data,
                N_rounds = N_rounds,
                N_alignment = N_alignment, 
                N_logits_matching_round = N_logits_matching_round,
                logits_matching_batchsize = logits_matching_batchsize, 
                N_private_training_round = N_private_training_round, 
                private_training_batchsize = private_training_batchsize)

initialization_result = fedmd.init_result
pooled_train_result = fedmd.pooled_train_result

collaboration_performance = fedmd.collaborative_training()

start model initialization: 
model  0
start full stack training ... 


2023-05-12 09:06:10.756620: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


full stack training done

model  1
start full stack training ... 


2023-05-12 09:06:14.498492: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/dropout_2/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


full stack training done

model  2
start full stack training ... 


...

model 7 done alignment
model 7 starting training with private data... 
model 7 done private training. 

model 8 starting alignment with public logits... 


2023-05-12 09:08:07.781305: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_18/dropout_19/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


model 8 done alignment
model 8 starting training with private data... 
model 8 done private training. 

model 9 starting alignment with public logits... 


2023-05-12 09:08:09.348092: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_19/dropout_22/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


model 9 done alignment
model 9 starting training with private data... 
model 9 done private training. 

round  1
update logits ... 
test performance ... 
0.1675
0.12020833333333333
0.08041666666666666
0.14270833333333333
0.2683333333333333
0.319375
0.35041666666666665
0.39958333333333335
0.508125
0.42520833333333335
[8, 9]
updates models ...
model 0 starting alignment with public logits... 
model 0 done alignment
model 0 starting training with private data... 
model 0 done private training. 

model 1 starting alignment with public logits... 
model 1 done alignment
model 1 starting training with private data... 
model 1 done private training. 

model 2 starting alignment with public logits... 
model 2 done alignment
model 2 starting training with private data... 
model 2 done private training. 

model 3 starting alignment with public logits... 
model 3 done alignment
model 3 starting training with private data... 
model 3 done private training. 

model 4 starting alignment with public l